<a href="https://colab.research.google.com/github/navidadkhah/Fine-Tuning-LLMs/blob/main/LLM/Fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [130]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [121]:
# Load the dataset to inspect its structure and content
file_path = '/content/drive/MyDrive/Bachelor\'s project/bug_evaluation_dataset.csv'
dataset = pd.read_csv(file_path)

# Display the first few rows of the dataset
dataset.head()

,original_code,modified_code,changed_line,number_of_line,mutation_type
0,n = int(input())\nw = [2] * n\nans0 = n\nans =...,n = int(input())\nw = [2] * n\nans0 = n\nans =...,"for j in range(y-1, r):",14,statement
1,n = int(input())\nw = [2] * n\nans0 = n\nans =...,n = int(input())\nw = [2] * n\nans0 = n\nans =...,if k<1:,8,decision
2,n = int(input())\nw = [2] * n\nans0 = n\nans =...,n = int(input())\nw = [2] * n\nans0 = n\nans =...,if k<1:,8,decision
3,n = int(input())\nw = [2] * n\nans0 = n\nans =...,n = int(input())\nw = [2] * n\nans0 = n\nans =...,"for j in range(l&1, r):",14,operation
4,n = int(input())\nw = [2] * n\nans0 = n\nans =...,n = int(input())\nw = [2] * n\nans0 = n\nans =...,"l, r, k = list(z(int, input().split()))",7,statement


In [122]:
# Drop rows where input or output is NaN
preprocessed_data = preprocessed_data.dropna(subset=['input', 'output'])

# Convert 'input' and 'output' columns to list of strings
inputs_list = preprocessed_data['input'].tolist()
outputs_list = preprocessed_data['output'].tolist()
preprocessed_data.head()

,input,output
0,statement: n = int(input())\nw = [2] * n\nans0...,n = int(input())\nw = [2] * n\nans0 = n\nans =...
1,decision: n = int(input())\nw = [2] * n\nans0 ...,n = int(input())\nw = [2] * n\nans0 = n\nans =...
2,decision: n = int(input())\nw = [2] * n\nans0 ...,n = int(input())\nw = [2] * n\nans0 = n\nans =...
3,operation: n = int(input())\nw = [2] * n\nans0...,n = int(input())\nw = [2] * n\nans0 = n\nans =...
4,statement: n = int(input())\nw = [2] * n\nans0...,n = int(input())\nw = [2] * n\nans0 = n\nans =...


In [123]:
from transformers import T5Tokenizer

# Initialize the T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Tokenize the inputs and outputs
inputs = tokenizer(inputs_list, padding=True, truncation=True, return_tensors='pt')
outputs = tokenizer(outputs_list, padding=True, truncation=True, return_tensors='pt')


In [124]:
class CodeFixDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': self.labels['input_ids'][idx]
        }

# Prepare the dataset
dataset = CodeFixDataset(inputs, outputs)

In [107]:
# Initialize the model
model = T5ForConditionalGeneration.from_pretrained('t5-small')

In [126]:
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2)

In [127]:
path ='/content/drive/MyDrive/Bachelor\'s project/results'
training_args = TrainingArguments(
    output_dir=path,
    per_device_train_batch_size=8,
    num_train_epochs=20,
    save_steps=1000,
    save_total_limit=2,
    logging_dir='/content/drive/MyDrive/Bachelor\'s project/logs',
    logging_steps=100,  # Log every 100 steps
    evaluation_strategy='epoch',  # You can set to 'steps' or 'epoch' based on your preference
    report_to="none"  # If you don't want to use wandb, set this to "none"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [128]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred  # Unpack the tuple (logits, labels)

    # Debugging: print the types and shapes of logits and labels
    print(f"Logits type: {type(logits)}, Labels type: {type(labels)}")
    print(f"Logits shape: {logits.shape}, Labels shape: {labels.shape}")

    # Ensure logits are a tensor and take argmax over the logits
    logits = logits[0] if isinstance(logits, tuple) else logits  # If logits is a tuple, take the first element

    # Get the predicted class (max logit) for each example
    predictions = torch.argmax(logits, dim=-1)  # Get the predicted class (max logit)

    # Flatten the logits and labels for comparison (especially in sequence tasks)
    predictions = predictions.flatten()
    labels = labels.flatten()

    # Compute accuracy
    accuracy = (predictions == labels).float().mean().item()  # Compute accuracy
    return {'accuracy': accuracy}


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # Provide the evaluation dataset
)

In [129]:
# Start the training process
# API key: 83361196b1d3f271296e4ec1ad99d660a639328e
train_output = trainer.train()
print(train_output)

Epoch,Training Loss,Validation Loss
1,No log,0.279062
2,No log,0.265724
3,No log,0.256648
4,No log,0.249727
5,No log,0.243070
6,No log,0.237144
7,No log,0.230368
8,No log,0.224081
9,No log,0.218370
10,No log,0.212038


TrainOutput(global_step=20, training_loss=0.8243042945861816, metrics={'train_runtime': 175.1259, 'train_samples_per_second': 0.114, 'train_steps_per_second': 0.114, 'total_flos': 2706836029440.0, 'train_loss': 0.8243042945861816, 'epoch': 20.0})


In [131]:
def find_accuracy(dataset):

  model.eval()

  # Prepare the test data
  test_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=False)

  # Lists to store true labels and predictions
  all_preds = []
  all_labels = []

  # Disable gradient computation during evaluation for efficiency
  with torch.no_grad():
      for batch in test_loader:
          input_ids, labels = batch['input_ids'], batch['labels']

          # If you're using a sequence-to-sequence model (like T5), provide the input_ids and labels
          outputs = model(input_ids=input_ids, labels=labels)  # Pass both input_ids and labels
          logits = outputs.logits  # Access logits (the output of the model)

          # Get predictions (use argmax to get the most likely class)
          preds = torch.argmax(logits, dim=-1)

          # Flatten predictions and true labels for multi-class classification
          all_preds.extend(preds.cpu().numpy().flatten())  # Flatten the predictions
          all_labels.extend(labels.cpu().numpy().flatten())  # Flatten the labels

  return all_preds, all_labels

In [135]:
all_preds_train, all_labels_train = find_accuracy(train_dataset)
all_preds_test, all_labels_test = find_accuracy(test_dataset)

# Compute accuracy
accuracy = accuracy_score(all_labels_train, all_preds_train)
print(f"Train Accuracy: {accuracy_percentage:.2f}%")

accuracy = accuracy_score(all_labels_test, all_preds_test)
print(f"Test Accuracy: {accuracy_percentage:.2f}%")

Train Accuracy: 89.65%
Test Accuracy: 89.65%
